In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from collections import defaultdict

In [22]:
data_mappping = joblib.load('../data/oc20_data_mapping.pkl')

In [23]:
len(data_mappping)

1281041

In [24]:
file_names = [i for i in os.listdir('../data/s2ef/2M/train') if i.find('.txt')>=0]
new_dict = []
for file in file_names:
    path = os.path.join('../data/s2ef/2M/train', file)
    with open(path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        system_id, frame_id = line.split(',')
        temp = {}
        temp['system_id'] = system_id
        temp['frame_id'] = frame_id
        temp.update(data_mappping.get(system_id, {}))
        new_dict.append(temp)

In [25]:
df = pd.DataFrame(new_dict)

In [26]:
df.shape

(2000000, 13)

In [27]:
df['system_id'].duplicated().sum()

1427550

In [28]:
df[['system_id', 'frame_id']].duplicated().sum()

0

In [29]:
df.head()

,system_id,frame_id,bulk_id,ads_id,bulk_mpid,bulk_symbols,ads_symbols,miller_index,shift,top,adsorption_site,class,anomaly
0,random876497,frame324\n,3532,14,mp-1225059,Fe2Cl8,*CH2OH,"(2, 1, 2)",0.022,False,"((0.56, 5.11, 25.48),)",3,0
1,random1978966,frame40\n,2593,27,mp-11453,Pd12Hf4,*CHCHO,"(2, 0, 1)",0.147,False,"((3.88, 3.7, 24.34),)",0,0
2,random621127,frame182\n,7708,24,mp-10676,Al2Si4Sc4,*CCH2,"(1, 1, 1)",0.142,False,"((0.09, 2.43, 24.67),)",1,0
3,random1660476,frame146\n,986,34,mp-865357,Zn3Pd,*CHCH2,"(1, 0, 1)",0.125,False,"((-0.66, 3.28, 20.65),)",0,0
4,random1030128,frame65\n,130,4,mp-754628,W4N8,*C,"(0, 2, 1)",0.102,True,"((6.67, 2.64, 21.91),)",2,0


In [30]:
df['anomaly'].value_counts()

anomaly
0    1311710
1     421258
3     129934
2     113415
4      23683
Name: count, dtype: int64

In [31]:
df['class'].value_counts()

class
0    842342
2    620850
1    455210
3     81598
Name: count, dtype: int64

In [32]:
df[df['anomaly']==0]['class'].value_counts()

class
0    585390
2    381416
1    304439
3     40465
Name: count, dtype: int64

In [33]:
df[df['anomaly']==0]['ads_symbols'].value_counts()

ads_symbols
*CCO          31303
*CHCO         30297
*CCH          29009
*COCH3        28914
*OCH2CH3      27950
              ...  
*ONN(CH3)2     8823
*CHOHCHOH      6444
*CH4           2046
*NO2NO2        1039
*OHNNCH3        608
Name: count, Length: 67, dtype: int64

In [34]:
stat = df[df['anomaly']==0].groupby(['class', 'ads_symbols'])['system_id'].count().reset_index()

In [35]:
stat.shape

(268, 3)

In [36]:
def show(df):
    result = {}
    temp = df[df['system_id'] == df['system_id'].min()]
    temp['print'] = df[['ads_symbols', 'system_id']].apply(lambda x: (x['ads_symbols'], x['system_id']), axis=1)
    result['min'] = temp['print'].tolist()
    temp = df[df['system_id'] == df['system_id'].max()]
    temp['print'] = df[['ads_symbols', 'system_id']].apply(lambda x: (x['ads_symbols'], x['system_id']), axis=1)
    result['max'] = temp['print'].tolist()
    return result
stat.groupby('class').apply(show)

class
0    {'min': [('*OHNNCH3', 86)], 'max': [('*CCO', 1...
1    {'min': [('*OHNNCH3', 38)], 'max': [('*CCO', 7...
2    {'min': [('*OHNNCH3', 269)], 'max': [('*CCO', ...
3    {'min': [('*NO2NO2', 108)], 'max': [('*CCH', 9...
dtype: object

In [37]:
stat.sort_values(['ads_symbols', 'class']).reset_index(drop=True)

,class,ads_symbols,system_id
0,0,*C,4325
1,1,*C,2642
2,2,*C,4592
3,3,*C,451
4,0,*C*C,9120
...,...,...,...
263,3,*ONNH2,538
264,0,CH2*CO,11125
265,1,CH2*CO,5386
266,2,CH2*CO,6062


In [38]:
def show(df):
    result = {}
    df = df['bulk_symbols'].value_counts().to_frame().reset_index()
    df.columns = ['bulk_symbols', 'system_id']
    df = df.sort_values('system_id')
    temp = df[df['system_id'] == df['system_id'].min()]
    temp['print'] = df[['bulk_symbols', 'system_id']].apply(lambda x: (x['bulk_symbols'], x['system_id']), axis=1)
    result['min'] = temp['print'].tolist()[:3]
    temp = df[df['system_id'] == df['system_id'].max()]
    temp['print'] = df[['bulk_symbols', 'system_id']].apply(lambda x: (x['bulk_symbols'], x['system_id']), axis=1)
    result['max'] = temp['print'].tolist()[:3]
    return result
df[df['ads_symbols']=='*C'].groupby('class').apply(show)

class
0    {'min': [('Ca2CdHg', 1), ('TaNbMo', 1), ('SrZn...
1    {'min': [('Sr4Cd2As4', 1), ('CaGe2Rh2', 1), ('...
2    {'min': [('Si2Se2Hf2', 1), ('P16S12', 1), ('Fe...
3    {'min': [('Rb2PdCl4', 1), ('In2Pb2Cl6', 1), ('...
dtype: object